In [8]:
'''

Una vez que toda la data es consumible por la API, está lista para consumir por los departamentos de Analytics y Machine Learning, y nuestro EDA nos permite entender bien los datos a los que tenemos acceso, es hora de entrenar nuestro modelo de machine learning para armar un sistema de recomendación. Para ello, te ofrecen dos propuestas de trabajo: En la primera, el modelo deberá tener una relación ítem-ítem, esto es se toma un item, en base a que tan similar esa ese ítem al resto, se recomiendan similares. Aquí el input es un juego y el output es una lista de juegos recomendados, para ello recomendamos aplicar la similitud del coseno. La otra propuesta para el sistema de recomendación debe aplicar el filtro user-item, esto es tomar un usuario, se encuentran usuarios similares y se recomiendan ítems que a esos usuarios similares les gustaron. En este caso el input es un usuario y el output es una lista de juegos que se le recomienda a ese usuario, en general se explican como “A usuarios que son similares a tí también les gustó…”. Deben crear al menos uno de los dos sistemas de recomendación (Si se atreven a tomar el desafío, para mostrar su capacidad al equipo, ¡pueden hacer ambos!). Tu líder pide que el modelo derive obligatoriamente en un GET/POST en la API símil al siguiente formato:

Si es un sistema de recomendación item-item:

def recomendacion_juego( id de producto ): Ingresando el id de producto, deberíamos recibir una lista con 5 juegos recomendados similares al ingresado.

En este Ipynb haremos el modelo de machine learning y en el main.py haremos el GET/POST.

'''

'\n\nUna vez que toda la data es consumible por la API, está lista para consumir por los departamentos de Analytics y Machine Learning, y nuestro EDA nos permite entender bien los datos a los que tenemos acceso, es hora de entrenar nuestro modelo de machine learning para armar un sistema de recomendación. Para ello, te ofrecen dos propuestas de trabajo: En la primera, el modelo deberá tener una relación ítem-ítem, esto es se toma un item, en base a que tan similar esa ese ítem al resto, se recomiendan similares. Aquí el input es un juego y el output es una lista de juegos recomendados, para ello recomendamos aplicar la similitud del coseno. La otra propuesta para el sistema de recomendación debe aplicar el filtro user-item, esto es tomar un usuario, se encuentran usuarios similares y se recomiendan ítems que a esos usuarios similares les gustaron. En este caso el input es un usuario y el output es una lista de juegos que se le recomienda a ese usuario, en general se explican como “A us

In [18]:
#Agregamos las librerias necesarias
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import sklearn as skl
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.decomposition import TruncatedSVD

In [10]:
#Importamos el parquet 

df = pd.read_parquet('df_useritemsgames.parquet')

In [14]:
#Ver los tipos de datos de las columnas

df.dtypes

user_id                object
funny                  object
posted                 object
last_edited            object
item_id                 int64
recommend                bool
sentiment_analysis      int64
items_count             int64
steam_id                int64
item_name              object
playtime_forever      float64
playtime_2weeks       float64
publisher              object
genres                 object
title                  object
release_date           object
tags                   object
specs                  object
price                 float64
early_access             bool
id                      int64
developer              object
year                    int64
MoneySpend            float64
contenido_free           bool
dtype: object

In [21]:
#Empezamos la creacion del modelo de machine learning para recomendacion de juegos
# Seleccionar columnas relevantes
df = df[['id', 'title']]

# Eliminar duplicados
df = df.drop_duplicates(subset='id')

# Llenar valores nulos
df = df.dropna()

# Crear una matriz TF-IDF para los títulos de los juegos
tfidf_vectorizer = TfidfVectorizer(stop_words='english')
tfidf_matrix = tfidf_vectorizer.fit_transform(df['title'])

# Calcular la similitud del coseno entre los juegos
cosine_sim = cosine_similarity(tfidf_matrix, tfidf_matrix)

In [22]:
# Crear un mapeo inverso de índices y títulos de juegos

indices = pd.Series(df.index, index=df['title']).drop_duplicates()

# Función que obtiene el top 10 de juegos más similares

def get_recommendations(title, cosine_sim=cosine_sim):
    # Obtener el índice del juego que coincide con el título
    idx = indices[title]

    # Obtener las puntuaciones de similitud entre juegos
    sim_scores = list(enumerate(cosine_sim[idx]))

    # Ordenar los juegos según las puntuaciones de similitud
    sim_scores = sorted(sim_scores, key=lambda x: x[1], reverse=True)

    # Obtener las puntuaciones de los 10 juegos más similares
    sim_scores = sim_scores[1:11]

    # Obtener los índices de los juegos
    game_indices = [i[0] for i in sim_scores]

    # Devolver los 10 juegos más similares
    return df['title'].iloc[game_indices]


In [23]:
#Lectura del df

df

,id,title
0,10,Counter-Strike
16302,20,Team Fortress Classic
32027,30,Day of Defeat
47071,50,Half-Life: Opposing Force
58006,300,Day of Defeat: Source
...,...,...
6081336,260250,Blood of the Werewolf
6081406,273070,The Last Federation
6081473,420740,The Deed
6081555,310850,Onikira - Demon Killer
